In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import random 
import json 
import math

In [73]:
from lib.local_database import Database, Collection, Question, db

from lib.html_viewer import view_questions as view
from lib.html_viewer import view_components as view_comp
from lib.html_viewer import extract_vectorimage as extract
from lib.html_viewer import svg_to_component as compose
from lib.html_viewer import tabl,trow,telm,underline,inpt,mini_red
from lib.html_viewer import h3,p,empty,span,div,mkrow,mktabl

from lib.svg_generator import diagram

In [78]:
from lib.svg_generator import *

## setup

In [80]:
winkel_tip = h3("beim Gegenwinkel sind alpha und beta gleich groß:")

winkel_tip += f'''<div style="background:white;display:inline">  {svg(
        
        line((900,500),(100,500),),
        line((800,800),(200,200),),

        line(*bow((500,500),200,270,315)),
        line(*bow((500,500),200,90,135)),



        text(400,470,"α"),
        text(600,550,"β"),

        text(150,900,"α = β ")

        
        )}</div>'''


winkel_tip += h3("beim Nebenwinkel  ergeben α und β zusammen 180° :")

winkel_tip += f'''<div style="background:white;display:inline">  {svg(
        
        line((900,500),(100,500),),
        line((800,800),(200,200),),

        line(*bow((500,500),200,270,315)),
        line(*bow((500,500),120,315,90)),



        text(400,470,"α"),
        text(500,450,"β"),

        text(150,900,"α + β = 180°"),
        text(150,940,"α = 180° - β")


        
        )}</div>'''




view_comp(winkel_tip)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [74]:
collection:Collection = db.get_collection("math","msa")

In [6]:
collection

<Collection math_msa 0 types>

In [82]:

def comp(deg:float):
    html = f'''<div style="background:white;display:inline">  {svg(
        
        line(
            (800,700),
            (250,200),
            (150,500),
            (850,500)),

        line(*bow((580,500),200,270,310)),
        line(*bow((580,500),200,90,130)),
        line(*bow((250,200),140,135,195)),


        text(450,470,"α"),
        text(650,550,f"β={deg}°"),
        text(230,300,"γ=63°"),

        text(150,900,"Zeichnung ist nicht Maßstabsgetreu")
        
        )}</div>'''

    return html

view_comp(comp(30))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [83]:
def gen(deg:float):
    c = comp (deg)
    c += p("wie groß ist beta?")
    return Question.standart(c,deg,[],tip=winkel_tip)

view(gen(40))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [84]:
q = [gen(d) for d in range(20,50,3)]

In [85]:
view(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


### anteil

In [128]:
def comp(pieces=[]):

    rot = random.randint(0,360)

    html = f'''<div style="background:white;display:inline">  {svg(
        
        line(*bow((500,500),300,0,360)),
        line((500,500,),*bow((500,500),300,0   + rot, 60  + rot),(500,500),fill = "grey" if 60  in pieces else "white"),
        line((500,500,),*bow((500,500),300,60  + rot, 180 + rot),(500,500),fill = "grey" if 120 in pieces else "white"),
        line((500,500,),*bow((500,500),300,240 + rot, 270 + rot),(500,500),fill = "grey" if 30 in pieces else "white"),
        line((500,500,),*bow((500,500),300,270 + rot, 360 + rot),(500,500),fill = "grey" if 90 in pieces else "white"),

        text(150,900,"Zeichnung ist nicht Maßstabsgetreu"),
        
        )}</div>'''

    return html

view_comp(comp([120,90]))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [150]:
def gen(pieces = []):

    possibs = ["1/2","1/3","1/4","1/6","1/12"]

    corrects=  list(map(lambda p: f"1/{int(360/p)}",pieces))

    for x in corrects:
        assert x in possibs ,f"illegal option {x}"

    wrongs = list(filter(lambda p:p not in corrects,possibs))

    return Question.multiple_choice(comp(pieces)+p("welcher teil ist markiert?"),corrects,wrongs,"")

view(gen([120,30]))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [153]:
q = [
    gen([30,120]),
    gen([120,90]),
    gen([90,60]),
    gen([60,30]),
]

In [154]:
view(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [155]:
collection.add_series(q,5,"anteile erkennen")

### potenzen vergleichen 

In [158]:
def gen(*item):

    nums = map (lambda (x,y):x**y,item)

    mx = max(item)
    wrongs = list(filter( lambda x: x != mx, item))

    return Question.multiple_choice(h3("welches ist die größte Zahl?"),[mx],wrongs,"")


In [159]:
gen((1,2),(2,1))

<Q:  <h3><h3>welches ist die größte Zahl?</h3 : ["(2, 1)"]>

## save

In [77]:
collection.save()

sname: gegenwinkel rstep: 0.5
